### Baudouin M. RAMAZANI

#### PHYS60

The point of this project is to learn how to separately apply the kinetic and potential parts of the hamiltonian operators so as to evolve a wavefunction in time. The context of this exercise is the penetration of a barrier by a wave packet, but the technique can be applied to many other situations of interest.


In [6]:
#
# cp7: Starter for Computing Project 7: FFT evolution with potential energy
#

from vpython import *
from numpy import *
from vpython import rate
inv = linalg.inv


In [7]:
canvas(width=600, height=300)
gd = graph(title="P(left) and P(right) vs. t", xtitle="t", ytitle="Probability",
          width=600, height=200)

def SetArrowFromCN( cn, a):
    """
    SetArrowWithCN takes a complex number  cn  and an arrow object  a .
    It sets the  y  and  z  components of the arrow s axis to the real 
    and imaginary parts of the given complex number. 
    
    Just like Computing Project 1, except y and z for real/imag.
    """
    a.axis.y = cn.real
    a.axis.z = cn.imag
    a.axis.x = 0.0
    

gL = gcurve(color=color.blue)
gR = gcurve(color=color.green)

hbar=1.0                       # use units where hbar = 1
m=1.0                          # and m=1.0
NA=500                         # how many arrows?
NA2=int(NA/2)                  # half of NA
b=30.0                         # range of x is -b/2 to b/2
a=0.01*b                       # half size of barrier (using Griffiths' notation)
V0=10.0                        # height of potential barrier

x = linspace(-b/2, b/2, NA)    # NA locations from -b/2 to b/2
n = arange(NA)                 # n = array([0,1,2,3,.... N-1])
n = piecewise(n, [n<NA2, n>=NA2], [lambda n:n, lambda n:n - NA])
                               # adjust n to give correct states with k<0
k = 2*n*pi/b
Energy = (hbar*k)**2/(2.0*m)   # get the KE array (one element for each k)
omega = Energy/hbar            # get the corresponding frequency

t = 0.0
dt = 0.005
kMin = 2*pi/b                  # lowest possible wave number
k0 = 20*kMin                   # nice packet central wave number
sigma = b/15.0                 # make packet fairly narrow
arrowScale = sqrt(NA*b*sigma)/10.0 # arbitrary scaling factor

psi=exp(1j*k0*x - ((x+1*b/4)/sigma)**2)   # gaussian wave packet
psi = psi/sqrt((abs(psi)**2).sum())       # normalize it!

def GetTransAmpl(kVal):
    """
    compute the transmission amplitude for an incoming wave with
    wavenumber 'kVal'. 
    """
    E = (hbar*kVal)**2/(2*m)                # Get the energy for this k
    kappa = sqrt(2.0*m*(V0*(1+0j)-E))/hbar  # Get the corresponding kappa
    
    #
    # set up the array to handle the boundary conditions for the 
    # wavefunction and its derivatives at the square barrier
    # boundaries (-a, +a).
    #
    
    M = array(
    [[-exp(1j*kVal*a), exp(-kappa*a), exp(kappa*a), 0.0],
     [1j*kVal*exp(1j*kVal*a), kappa*exp(-kappa*a), -kappa*exp(kappa*a), 0.0],
     [0.0, exp(kappa*a), exp(-kappa*a), -exp(1j*kVal*a)],
     [0.0, kappa*exp(kappa*a), -kappa*exp(-kappa*a), -1j*kVal*exp(1j*kVal*a)]
     ])
     
    B = array(
    [[exp(-1j*kVal*a)],
     [1j*kVal*exp(-1j*kVal*a)],
     [0.0],
     [0.0],
     ])

    return (asmatrix(inv(M))*B)[-1,0]



#
# Compute the weighted average of the transmission probability
# for each k in the fourier transform weighted by the probability
# of that "k" being measured in the incoming wave packet.
#

sum=0.0
wt=0.0
phi = fft.fft(psi)


for i in range(NA2):
    trans = abs(GetTransAmpl(k[i]))**2
    phiSQ = abs(phi[i])**2
    sum += phiSQ*trans
    wt += phiSQ

print("Theoretical Transmission Probability: T=", sum/wt)

#
# set up potential.
#

V = zeros(len(psi),double)
beg=NA2 - (int(round(NA*a/b)))
end=NA2 + (int(round(NA*a/b)))
V[beg:end] =V0*ones(end-beg,double)

#
# Set up the 3D representation of the barrier
#

barrier = cylinder(pos=vec(x[beg],0,0), axis=vec(x[end]-x[beg],0,0), color=color.blue, opacity=0.4, radius=3.0)

#
# set up the arrows
#

alist = []
for i in range(NA):
    alist.append(arrow(pos=vec(x[i],0,0), axis=vec(0,1,0), color=color.red))
    SetArrowFromCN(arrowScale*psi[i],alist[i])
    
maxT=0.0
while True:
    rate(1.0/dt)
    #
    # Here's where you put in the code to use the FFT/IFFT
    # to propagate the state forward in time and graph
    # the probability of finding the particle on the 
    # left or right of the potential barrier.
    #
    
    phi=fft.fft(psi)
    psi=fft.ifft(exp(-1j*(omega)*dt)*phi)
    psi=psi*exp(V*dt*-1j)
    i = 0
    while i< NA:
        SetArrowFromCN(arrowScale*psi[i], alist[i])
        i += 1
    
    y1 = (abs(psi[NA2:])**2).sum()
    y2 = (abs(psi[:NA2])**2).sum()
    gL.plot(t,y2)
    gR.plot(t,y1)
    
    t+=dt
    
    pass

<IPython.core.display.Javascript object>

Theoretical Transmission Probability: T= 0.2971150726971718


KeyboardInterrupt: 

1) If you increase the barrier height from 10 units to 100 units you will find that you need to reduce the time step to maintain reasonable results. why?

  ####  It decreases the amplitude of passing through the barrier.

2) Why does the proprability graph display nonsense after about 3.6 time units with the initial conditions provided in the starter program?


#### It oscillates between the left and right of the barrier and the probability remain the same in every case.

3) How could you modify the program so that it would compute reasonable probabilities for later times? Which initial conditions would you need to modify in the program to make that work?

 #### Run the program with a paricle that has a smaller momentum.